In [8]:
import pandas as pd
from sklearn.decomposition import NMF
from utils.process import prepare_df, data_arrays, data_tensors, LABELS
from utils.logging import log
from sklearn import metrics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from ssnmf.ssnmf import SSNMF_T
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.cluster import KMeans


In [9]:
df = prepare_df()
df = df[df['Label'] != 'BENIGN']
df = df.sample(1000)
df.groupby('Label')['Label'].count()

Label
Bot                   1
DDoS                237
DoS GoldenEye        19
DoS Hulk            396
DoS Slowhttptest      6
DoS slowloris        10
FTP-Patator          22
Infiltration          1
PortScan            295
SSH-Patator          11
Web Attack   XSS      2
Name: Label, dtype: int64

In [10]:
from ssnmf import SSNMF
import random

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


def get_Y_torch(X, y):
    y_unique = torch.unique(y)
    sample_size = X.shape[1]
    Y = torch.zeros(y_unique.shape[0], sample_size, device=device)
    for i in range(sample_size):
        j = ((y_unique == y[i]).nonzero(as_tuple=True)[0])
        Y[j, i] = 1
    return Y


def get_L_torch(Y, fraction_known=0.5):
    L = torch.zeros(Y.shape, device=device)
    m, n = L.shape
    num_samples = int(fraction_known * n)
    labeled_data = torch.randperm(n, dtype=torch.int32, device=device)[:num_samples]
    for i in labeled_data:
        L[:, i] = torch.tensor(1)
    return L

#model = SSNMF(M_s,10, modelNum=1)

In [ ]:
X, y = data_tensors(df)

skf = StratifiedKFold(n_splits=5)


for fraction_known in [0.1, 0.3, 0.5, 0.6, 0.8]:
    for k in [5, 10, 20, 30]:
        for lam in (0, 0.001, 0.5, 0.1, 0.5, 1, 10, 100, 500, 1000, 5000):
            for train_index, test_index in skf.split(X, y):
                X_train_T, X_test_T = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                Y_train = get_Y_torch(X_train_T.T, y_train)
                L_train = get_L_torch(Y_train, fraction_known=fraction_known)
                #print(f"norm X {np.linalg.norm(X_train_T)}")
                snmf = SSNMF(X_train_T.T, k, L=L_train, Y=Y_train, lam=lam * np.linalg.norm(X_train_T), modelNum=5)
                snmf.mult(numiters=1000)
                y_pred = np.argmax(snmf.S, axis=0)
                kmeans = KMeans(n_clusters=k, random_state=0).fit(snmf.S.T)
                #rand_score = metrics.rand_score(y_train, kmeans.labels_)
                adujsted_rand_score = metrics.adjusted_rand_score(y_train, kmeans.labels_)

                #a_mutual_score = metrics.adjusted_mutual_info_score(y_train, kmeans.labels_)
                #print(f"Rand score {rand_score}")
                #print(f"Adjusted rand score {adujsted_rand_score}")
                #print(f"Adjusted mutual score {a_mutual_score}")
                results = [fraction_known, k, lam, adujsted_rand_score]
                log(results)




/home/tosia/NASK/network_analyze_new/utils/process.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)
/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


79
79
79
79
79
79


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


79
79
79
79
79
79


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


...

239
239
239
239
239
239


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


239
239
239
239
239
239


/home/tosia/.pyenv/versions/3.9.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


239
239
239
239
